In [3]:

import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of genai",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

I believe you meant to ask about the importance of "genetics" rather than "genai".

Genetics is the study of how traits are passed down from one generation to the next. The importance of genetics cannot be overstated, as it plays a crucial role in our understanding of many biological processes and has numerous applications in various fields. Here are some reasons why genetics is important:

1. **Understand disease susceptibility**: Genetics helps us understand the causes of many diseases, such as genetic disorders, and can inform strategies for prevention and treatment.
2. **Development and evolution**: Genetics explains how species adapt to their environments and evolve over time, which is essential for understanding the diversity of life on Earth.
3. **Cancer research**: Genetic mutations linked to cancer can help researchers develop targeted therapies and improve treatment options.
4. **Genetic counseling**: Understanding genetic traits can help families make informed decisions abou

In [ ]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

def ask_question(question, model="llama3-8b-8192"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

# Main loop to ask questions
while True:
    question = input("Enter your question (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        break
    answer = ask_question(question)
    print(answer)


In [2]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

def ask_question(question, model="llama3-8b-8192"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

def get_multiline_input(prompt):
    print(prompt)
    buffer = []
    while True:
        line = input()
        if line.strip().lower() == 'exit':
            return 'exit'
        if line == "":
            break
        buffer.append(line)
    return "\n".join(buffer)

# Main loop to ask questions
try:
    while True:
        question = get_multiline_input("Enter your question (press Enter on an empty line to finish, type 'exit' to quit): ")
        if question.strip().lower() == 'exit':
            break
        print(f"Question: {question}")
        answer = ask_question(question)
        print(f"Answer: {answer}")
except KeyboardInterrupt:
    pass
finally:
    print("\nExiting...")


Enter your question (press Enter on an empty line to finish, type 'exit' to quit): 


Question: i want some information about some products listed in amazon like product name, price and rating so can you provide me with that ?
Answer: I'd be happy to help you with that. However, I'm a large language model, I don't have direct access to Amazon's database, but I can assist you in finding the information you're looking for.

Please provide me with the product names or ASINs (Amazon Standard Identification Number) of the products you're interested in, and I'll do my best to fetch the following information:

1. Product Name
2. Current Price
3. Customer Rating (out of 5 stars)

Please note that prices and ratings may change over time, so this information may not be up-to-date. Additionally, Amazon's policies prohibit scraping or scraping content from their website, so I'll need to rely on Amazon's public APIs or cached versions of product pages.

Please provide the product names or ASINs, and I'll do my best to retrieve the information you're looking for.
Enter your question 

In [13]:
import os
import asyncio
import time
import requests
import pandas as pd
import nest_asyncio
from bs4 import BeautifulSoup
from scrapegraphai.graphs import SearchGraph

# Apply nest_asyncio to handle nested event loops
nest_asyncio.apply()

# Define the configuration for the graph
graph_config = {
    "llm": {
        "model": "groq/llama3-8b-8192",
        "api_key": os.environ.get("GROQ_API_KEY"),  # Fetch the API key from environment variables
        "temperature": 0
    },
    "embeddings": {
        "model": "ollama/nomic-embed-text",
        "base_url": "http://localhost:11434",  # Ensure this URL is correct and the service is running
    },
    "max_results": 5,
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

# Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'}).string.strip()
    except AttributeError:
        try:
            price = soup.find("span", attrs={'id': 'priceblock_dealprice'}).string.strip()
        except:
            price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
        except:
            rating = ""
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""
    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"
    return available

# Function to extract Product Image URL
def get_image_url(soup):
    try:
        image_container = soup.find("div", attrs={'class': 'imgTagWrapper'})
        img_tag = image_container.find('img')
        if img_tag and 'src' in img_tag.attrs:
            image_url = img_tag['src']
        else:
            image_url = ""
    except AttributeError:
        image_url = ""
    return image_url

# Save the webpage content to an HTML file with retry mechanism
def save_webpage_to_html(url, filename, retries=3, delay=5):
    for attempt in range(retries):
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(response.text)
            return
        elif response.status_code == 429:
            print(f"Rate limited. Retrying in {delay} seconds...")
            time.sleep(delay)
        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            break
    raise Exception("Failed to retrieve the webpage after several attempts")

# Parse the HTML file and extract information
def extract_information_from_html(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    products = []
    for product in soup.select('div.s-main-slot div.s-result-item'):  # Adjust the selector based on the actual HTML structure
        title = get_title(product)
        price = get_price(product)
        rating = get_rating(product)
        review_count = get_review_count(product)
        availability = get_availability(product)
        image_url = get_image_url(product)
        
        if title:
            products.append({
                'name': title,
                'price': price,
                'rating': rating,
                'reviews': review_count,
                'availability': availability,
                'image_url': image_url
            })
    
    return products

async def run_search_graph():
    try:
        # Save the webpage content to an HTML file
        url = "https://www.amazon.com/s?k=iphone+15+pro+max&ref=nb_sb_noss_1"
        filename = "webpage.html"
        save_webpage_to_html(url, filename)
        
        # Extract information from the saved HTML file
        products = extract_information_from_html(filename)
        df = pd.DataFrame(products)
        print(df)
    except requests.exceptions.RequestException as req_e:
        print(f"Request error: {req_e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Main function to run the async task
async def main():
    await run_search_graph()

# Execute the main function
if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # If loop is already running, use nest_asyncio to run the coroutine
            nest_asyncio.apply()
            loop.run_until_complete(main())
        else:
            loop.run_until_complete(main())
    except RuntimeError as e:
        print(f"RuntimeError: {e}")
        # Create a new event loop if necessary
        new_loop = asyncio.new_event_loop()
        asyncio.set_event_loop(new_loop)
        new_loop.run_until_complete(main())


Failed to retrieve the webpage. Status code: 503
An error occurred: Failed to retrieve the webpage after several attempts
